In [18]:
# from huggingface_hub import snapshot_download

repo_id = 'allenai/pixmo-cap'
# target_dir = '/lambdafs/jacob/data/' + repo_id.replace('/', '_')

from datasets import load_dataset
dataset = load_dataset(repo_id, split='train', streaming=True)

# snapshot_download(repo_id=repo_id, local_dir=target_dir, repo_type='dataset')


In [1]:
%load_ext autoreload
%autoreload 2

# import nest_asyncio
# nest_asyncio.apply()

In [2]:
from data_manager import DataManager

In [3]:
manager = DataManager(db_path='/home/jacob/Desktop/test_db.db', download_manager_workers=300)

In [5]:
manager.process_dataset("data_sets/pixmo-cap.py", "/home/jacob/Desktop/data")

Processing Rows: 10000rows [00:00, 10842.58rows/s]


In [ ]:
# manager.download_manager.get_stats()
manager.download_manager.get_failed_downloads()

In [11]:
data = manager.sample_data(data_sources=['pixmo-cap'], limit=100, offset=20)
data[0]

{'uuid': '43f900ec-8bb3-4b64-b58d-624e2df177a6',
 'data_source': 'pixmo-cap',
 'group_id': 'pixmo-cap-20241206',
 'data_type': 'default',
 'data': "The image depicts an upscale Infiniti car dealership showroom, characterized by its spacious and well-lit interior. The showroom features dark brown wooden floors and a large area of beige-ish, white-ish tile where five Infiniti cars are neatly lined up, three on the right and two on the left. The cars are in various colors, including blue, silver, white, red, and gray. The center of the showroom has a consultation area furnished with dark brown leather couches, ideal for potential buyers to sit and discuss. A display case is also visible near this seating area. At the very back of the showroom, a counter is visible with the Infiniti logo displayed prominently. The building's walls are primarily made of glass, allowing abundant natural light to flood the space and offering views of the bright, sunny day outside. No people or written text ar

In [36]:
manager.delete_group(group_id="testing")

0